In [2]:
from toolbar.TN_WaveActivityFlux import TN_WAF_3D
from toolbar.curved_quivers.modplot import Curlyquiver
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature

#以下为数据读取部分，最终所得z300，u300，v300为2014年11月的300hPa位势高度场，UV风场的月平均；
#z_tmp，u_tmp，v_tmp为1979-2018年11月的气候态；
#所有数据来自ECMWF的ERA-Interim数据集
f_z300 = xr.open_dataset("E:\data\ERA5\ERA5_pressLev\era5_pressLev.nc").sel(date=slice(19790101, 20181231),
                                                                            pressure_level=300)
f_z300 = xr.Dataset({
    'z': (['date', 'level', 'lat', 'lon'], f_z300['z'].data[:, np.newaxis, :, :]),
    'u': (['date', 'level', 'lat', 'lon'], f_z300['u'].data[:, np.newaxis, :, :]),
    'v': (['date', 'level', 'lat', 'lon'], f_z300['v'].data[:, np.newaxis, :, :]), },
    coords={'date': pd.to_datetime(f_z300['date'], format="%Y%m%d"),
            'level': [300],
            'lat': f_z300['latitude'].data,
            'lon': f_z300['longitude'].data})
z300 = f_z300['z'].sel(date='2014-11-01')
z300 = z300
z_tmp = f_z300['z'].loc[f_z300.date.dt.month.isin([11])]
z_tmp = z_tmp.mean(('date'))
u300 = f_z300['u'].sel(date='2014-11-01')
u_tmp = f_z300['u'].loc[f_z300.date.dt.month.isin([11])]
u_tmp = u_tmp.mean(('date'))
v300 = f_z300['v'].sel(date='2014-11-01')
v_tmp = f_z300['v'].loc[f_z300.date.dt.month.isin([11])]
v_tmp = v_tmp.mean(('date'))
lat = f_z300['lat']
lon = f_z300['lon']


KeyboardInterrupt: 

In [ ]:
waf_x, waf_y, stream = TN_WAF_3D(z_tmp, u_tmp, v_tmp, z300 - z_tmp, return_streamf=True, filt=0)


In [ ]:
#以下为绘图部分
fig = plt.figure(figsize=(12, 6), dpi=600)
ax = fig.add_subplot(111, projection=ccrs.PlateCarree(central_longitude=180))
ax.add_feature(cfeature.COASTLINE.with_scale('110m'), linewidth=0.2)
WAF = Curlyquiver(ax, lon, lat, waf_x, waf_y, regrid=30, lon_trunc=0, arrowsize=1, scale=30, linewidth=1.4,
              color='black', transform=ccrs.PlateCarree(central_longitude=0), mode='loose')
WAF.key(fig, U=15, label='15 m$^2$/s$^2$', lr=1)
#ax_q = ax.quiver(lon, lat, waf_x, waf_y, regrid_shape=20, transform=ccrs.PlateCarree(central_longitude=0),scale=300)
ax_c = ax.contour(lon, lat, stream, levels=100, transform=ccrs.PlateCarree(central_longitude=0), colors='black')
plt.show()